# Local Image Generation with Stable Diffusion (Long Prompt + Img2Img + Negative Prompt)
This notebook sets up a local image-to-image and text-to-image tool. It is designed to be beginner-friendly with detailed explanations for every single line of code.

**Key Technical Concepts:**
- **Stable Diffusion:** An AI model that creates images from text.
- **LPW (Long Prompt Weighting):** A trick that lets you bypass the standard 77-word limit for prompts.
- **Img2Img:** Using an existing image as a "blueprint" for the AI to modify.
- **Negative Prompt:** Telling the AI exactly what you *don't* want to see (e.g., "no tattoos", "not blurry").

In [ ]:
# Step 1: Install the necessary software packages (dependencies)
# !pip is a command to download and install Python tools
# diffusers: The main library for running AI image models
# transformers: Helps the computer understand the text you type
# accelerate: Makes the generation process run faster on your hardware
# scipy: A package for math that helps with image processing
# ipywidgets: Tools to create buttons, sliders, and text boxes in the notebook
!pip install -q diffusers transformers accelerate scipy ipywidgets

In [ ]:
# Import the core engine 'torch' which handles all the AI math
import torch
# 'io' allows our program to read and write bytes (raw data)
import io
# 'PIL' (Python Imaging Library) is used to open, resize, and save pictures
from PIL import Image
# 'StableDiffusionPipeline' is the standard toolkit for generating images
from diffusers import StableDiffusionPipeline
# 'widgets' creates the interactive UI elements like buttons
import ipywidgets as widgets
# 'display' allows us to show images and controls in the notebook output area
from IPython.display import display

# Check if your computer has a high-speed 'graphics card' (GPU) available
# If yes, we use 'cuda' (fast); otherwise, we use 'cpu' (slower)
device = "cuda" if torch.cuda.is_available() else "cpu"
# Print out which part of the computer is doing the work
print(f"Using hardware: {device}")

In [ ]:
# Define which version of Stable Diffusion we want to download
model_id = "runwayml/stable-diffusion-v1-5"

# Download and load the AI model into memory
# we use custom_pipeline="lpw_stable_diffusion" so you can write long prompts
# we use torch_dtype=float16 to save memory and run faster if using a GPU
pipe = StableDiffusionPipeline.from_pretrained(
    model_id, 
    custom_pipeline="lpw_stable_diffusion",
    torch_dtype=torch.float16 if device == "cuda" else torch.float32
)

# Move the entire model onto the chosen hardware (either the GPU or CPU)
pipe = pipe.to(device)

# Confirmation message so you know the model is ready to use
print("AI Model loaded and ready for long prompts, images, and negative prompts!")

In [ ]:
# This is a helper function that takes your prompt, optional image, then creates the output
def generate_image(prompt, negative_prompt="", init_image=None, strength=0.75):
    # Check if the user uploaded an image to start with
    if init_image:
        # Print progress so you know what the AI is doing
        print(f"Applying prompt to your image... (Strength: {strength})")
        # Convert the picture to standard colors (RGB) and resize it to 512x512 pixels
        init_image = init_image.convert("RGB").resize((512, 512))
        # Use 'autocast' to handle high-precision math automatically
        with torch.autocast(device):
            # Create the new image using BOTH the prompt and the uploaded picture
            # We also pass 'negative_prompt' to tell the AI what to avoid
            image = pipe(
                prompt=prompt, 
                negative_prompt=negative_prompt, 
                image=init_image, 
                strength=strength
            ).images[0]
    # If no image was uploaded, just create something new from scratch
    else:
        print(f"Generating new image from your description...")
        # Run the AI with text prompts only (both positive and negative)
        with torch.autocast(device):
            image = pipe(prompt=prompt, negative_prompt=negative_prompt).images[0]
    
    # Give the finished picture back to whoever called this function
    return image

In [ ]:
# --- Creating the Interactive Dashboard ---

# Create a large text area where you can type what YOU WANT to see
prompt_input = widgets.Textarea(
    value='A beautiful sunset over a cyberpunk mountain range, digital art, highly detailed, vivid colors, neon lights',
    placeholder='What do you want to see?',
    description='Positive:',
    disabled=False,
    layout=widgets.Layout(width='80%', height='80px')
)

# Create a text area where you can type what YOU DO NOT WANT to see
negative_input = widgets.Textarea(
    value='tattoos, blurry, low quality, distorted face, extra limbs',
    placeholder='What should the AI avoid?',
    description='Negative:',
    disabled=False,
    layout=widgets.Layout(width='80%', height='80px')
)

# Create an HTML label to show character and token counts in real-time
status_label = widgets.HTML(value="<i>Type above to see stats...</i>")

# Create a button that lets you upload a picture from your computer
uploader = widgets.FileUpload(
    accept='image/*',  # Only allow picture files (png, jpg, etc.)
    multiple=False,    # Only allow one picture at a time
    description='Upload Image',
    layout=widgets.Layout(width='150px')
)

# Create a slider to control the 'Strength' (how much to change the original)
strength_slider = widgets.FloatSlider(
    value=0.75, 
    min=0.0, 
    max=1.0, 
    step=0.05, 
    description='Strength:',
    tooltip='0.0 = exact original, 1.0 = completely new image',
    layout=widgets.Layout(width='80%')
)

# A descriptive label for the slider
ui_info = widgets.HTML(value="<p style='color: gray; font-size: 0.9em; margin-left:10px;'>Strength only matters for uploaded images.</p>")

# Create the main blue button that triggers the image creation
button = widgets.Button(description="Generate Now!", button_style='primary', layout=widgets.Layout(width='200px', height='40px'))

# Create a blank area where we will display the results
output = widgets.Output()

# This special 'observer' function runs every time you type OR upload an image
def update_status(change):
    # We always pull the current text from the input box directly
    # This avoids errors when 'change' comes from the file uploader
    text = prompt_input.value
    
    # Roughly estimate tokens (AI words) based on text length
    estimated_tokens = len(text.split()) * 1.3 
    # Count how many individual letters/characters you typed
    char_count = len(text)
    
    # Create a simple message to show the user
    status = f"<b>Pos. Characters:</b> {char_count} | <b>Est. Tokens:</b> {int(estimated_tokens)}"
    # If it's over 77 tokens, warn them that the 'Long Prompt' feature is working
    if estimated_tokens > 77:
        status += " <span style='color: orange;'>(Long prompt mode active!)</span>"
    
    # If a file is currently in the uploader, add a green success message
    if uploader.value:
        status += " | <span style='color: green;'><b>Image Loaded</b></span>"
    
    # Update the status label text on the screen
    status_label.value = status

# Tell the boxes to watch for changes and call 'update_status' when they happen
prompt_input.observe(update_status, names='value')
uploader.observe(update_status, names='value')

# This function runs when you click the 'Generate Now!' button
def on_button_clicked(b):
    # Tell the output area to show our progress messages and final image
    with output:
        # Clear anything that was there from a previous run
        output.clear_output()
        # If the positive input is empty, show a warning and stop
        if not prompt_input.value.strip():
            print("Oops! You forgot to type a description of what you WANT.")
            return
        
        init_image = None
        # If the user uploaded a file, extract the actual picture from it
        if uploader.value:
            # Unpack the file data from the uploader code into a PIL Image
            uploaded_file = list(uploader.value.values())[0] if isinstance(uploader.value, dict) else uploader.value[0]
            content = uploaded_file['content']
            init_image = Image.open(io.BytesIO(content))
            print("Uploaded image detected!")

        # Run the generation function with BOTH the positive and negative prompts
        image = generate_image(
            prompt_input.value, 
            negative_prompt=negative_input.value, 
            init_image=init_image, 
            strength=strength_slider.value
        )
        
        # If we used an uploaded image, show the Before/After side-by-side
        if init_image:
            print("\n--- [Original Image] | [AI Result] ---")
            side_by_side = Image.new('RGB', (1024, 512))
            side_by_side.paste(init_image.resize((512, 512)), (0, 0))
            side_by_side.paste(image.resize((512, 512)), (512, 0))
            display(side_by_side)
        # If no image was used, just display the result
        else:
            display(image)

# Link the button to the function we just wrote
button.on_click(on_button_clicked)

# Arrange the widgets neatly on the screen for the user
display(prompt_input, negative_input, status_label, widgets.HBox([uploader, ui_info]), strength_slider, button, output)

# Run the status update once at the start so we see counts immediately
update_status(None)